### Instructions for activated the Jupyter kernel for the `cmip6hack-multigen` conda environment

In a Jupyterlab terminal, navigate to the `/cmip6hack-multigen/` folder and run the command:
```bash
source spinup_env.sh
```
which will create the `cmip6hack-multigen` conda environment and install it as a python kernel for jupyter.

Then, switch the kernel (drop-down menu in the top right hand corner) to cmip6hack-multigen and restart the notebook.

### Pre-process climate model output in GCS

This notebook uses [`intake-esm`](https://intake-esm.readthedocs.io/en/latest/) to ingest and organize climate model output from various model generations and resave their time-mean fields locally.

In [27]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%autoreload 2

In [32]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xs
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util
import preprocess as pp
import qc

In [2]:
varnames = ['tas', 'pr', 'psl']
timeslice = slice('1981', '2010')

In [3]:
ens_dict = pp.load_ensembles(varnames, timeslice=timeslice)
ens_dict

Loaded: variable_id ` tas ` from activity_id ` far `
Loaded: variable_id ` pr ` from activity_id ` far `
Loaded: variable_id ` psl ` from activity_id ` far `
Loaded: variable_id ` tas ` from activity_id ` sar `
Loaded: variable_id ` pr ` from activity_id ` sar `
Loaded: variable_id ` psl ` from activity_id ` sar `
Loaded: variable_id ` tas ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` psl ` from activity_id ` tar `
Skip TAR.MPIfM.MPIfM.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` tas ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` pr ` from activity_id ` cmip3 `
Skip CMIP3.CSIRO-QCCCE.csiro_mk3_5.historical.Amon.gn before datetime conflict.
Loaded: variable_id ` psl ` from activity_id ` cmip3 `
Load

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,


{'far': <xarray.Dataset>
 Dimensions:   (ensemble: 3, lat: 90, lon: 180, time: 3)
 Coordinates:
   * time      (time) datetime64[ns] 1985-01-01 1995-01-01 2005-01-01
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * ensemble  (ensemble) object 'UKTR-UKTR' 'GISS-GISS' 'GFDL-GFDL'
 Data variables:
     tas       (ensemble, time, lat, lon) float64 dask.array<chunksize=(1, 3, 90, 180), meta=np.ndarray>
     pr        (ensemble, time, lat, lon) float64 dask.array<chunksize=(1, 3, 90, 180), meta=np.ndarray>,
 'sar': <xarray.Dataset>
 Dimensions:   (ensemble: 8, lat: 90, lon: 180, time: 360)
 Coordinates:
   * ensemble  (ensemble) object 'CCSR-NIES-CCSR-NIES-01' ... 'CSIRO-CSIRO-01'
   * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ..

#### Extracting time-mean

In [4]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.mean, on_self=True, dim =['time'], keep_attrs=True)

In [5]:
ens_dict = util.dict_func(ens_dict, xr.Dataset.compute, on_self=True)

/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/srv/conda/envs/cmip6hack-multigen/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


#### Adding ensemble-mean to the ensemble

In [6]:
ens_dict = util.add_ens_mean(ens_dict)

In [7]:
ens_dict

{'far': <xarray.Dataset>
 Dimensions:   (ensemble: 4, lat: 90, lon: 180)
 Coordinates:
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * ensemble  (ensemble) object 'ens-mean' 'UKTR-UKTR' 'GISS-GISS' 'GFDL-GFDL'
 Data variables:
     tas       (ensemble, lat, lon) float64 233.0 233.0 232.9 ... 257.8 257.8
     pr        (ensemble, lat, lon) float64 7.9e-06 7.882e-06 ... 1.458e-05
 Attributes:
     name:     far,
 'sar': <xarray.Dataset>
 Dimensions:   (ensemble: 9, lat: 90, lon: 180)
 Coordinates:
   * lat       (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * lon       (lon) float64 1.0 3.0 5.0 7.0 9.0 ... 353.0 355.0 357.0 359.0
   * ensemble  (ensemble) object 'ens-mean' ... 'CSIRO-CSIRO-01'
 Data variables:
     tas       (ensemble, lat, lon) float64 233.9 233.8 233.7 ... 258.9 258.9
     pr        (ensemble, lat, lon) float64 8.34e-06 8.312e-06 ... 5.599e-

### Pre-process observational data products

In [44]:
era5 = pp.load_era("../data/raw/reanalysis/ERA5_mon_2d.nc", timeslice=timeslice, coarsen_size=2)

### Save interim files

In [46]:
interim_path = "../data/interim/"
era5.to_zarr(interim_path + "era5_timemean")

In [47]:
for key, ens in ens_dict.items():
    ens.to_zarr(interim_path + f"{key}_timemean")